* Regression problems pop up whenever we want to predict anumerical value.
* Such cases include predicting prices,predicting the length of stay for patient in say a hospital, forecastinf demand etc.
* To develop a model for predicting house prices, we need to get our hands on data, which includes sales price,area,number of rooms etc.
* This dataset is called a `training dataset` or `training set`, and each row that contains data corrresponding to one sale is called a data point or sample.
* The thing that we're trying to predict is called a *label* or *target*.
* The variables upon which the predictions are based are called `features or covariates`

In [1]:
!pip install d2l --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 6.5 MB/s eta 0:00:00


In [2]:
%matplotlib inline
import math
import time
import numpy as np
import torch
from d2l import torch as d2l

## 1.1 Basics

* In linear regression we first assume that the relationship between features `x` and target `y` is approximately linear.
* We superscrpits to enumerate samples and targets, subscripts to index coordinates.
* $x^{(i)}$ denotes the $i^{th}$ and $x_j^{(i)}$ denotes its $j^{th}$  coordinate.

# Model

* The assumption of linearity means that the expected value of the target (price) can be expressed as a weighted sum of the features (area and age):

* $price = w_{area}* area + w_{age}*age + b$

* Here $w_{area}$ and   $w_{age} $ are called *weights* and $b$ is called $bias$.
* The weights determine the influence of each feature on our prediction.
* The bias determines the value of the estimate when all features are zero.
* In machine learning, we usually work with high-dimensional datasets, where it is more convenient to employ compact linear algebra notation.
* When our inputs consists of $d$ features, we assign each an index (between 0 and d) and express our prediction $\hat{y}$ as:
   
   $\hat{y}$ = $w_1*x_1 + ...+ w_d*x_d + b $

* We can also express our model compactly via dot product between `w` and `x`.

  $\hat{y}$ = $w^{T}*x + b$

* The vector `x` in the above formula corresponds to the features of a single examples

In [3]:
##implementing a basic regression example
import numpy as np
x = np.arange(12)
y_true = np.arange(12,24)
b = 1.5
y_hat = []

# For a single-feature linear regression, 'w' should be a single scalar weight.
w = 1.0

for x_val in x: # Iterate through each feature value in x
  preds = w * x_val + b
  y_hat.append(preds)

print(y_hat)
print(f"Number of predictions: {len(y_hat)}")
print(f"True values: {y_true}")

[np.float64(1.5), np.float64(2.5), np.float64(3.5), np.float64(4.5), np.float64(5.5), np.float64(6.5), np.float64(7.5), np.float64(8.5), np.float64(9.5), np.float64(10.5), np.float64(11.5), np.float64(12.5)]
Number of predictions: 12
True values: [12 13 14 15 16 17 18 19 20 21 22 23]


## Loss function

* Say we want to measure how accurate our model performs like how good is it's performance.
* We use a function called a *loss function* to do this.
* Basically it quantifies the distance between `real` and `predicted` values of the target.
* For regression problems, the most common loss function is the $ squared error$.
* When our prediction for an example $i$ is $\hat{y}^{(i)}$ and corresponding true label is $y^{(i)}$, the squared error is given by:

$l^{(i)}(w, b) = \frac{1}{2} \left( \hat{y}^{(i)} - y^{(i)} \right)^2$

* The constant $1/2$ makes no real differencw but proves to be notatioanally convenient, since it cancels out when we take the derivative of the loss.

* To measure the quality of a model on the entire dataset of `n` samples we simply average the losses on the training set:

 $
 L(\mathbf{w}, b) = \frac{1}{n} \sum_{i=1}^{n} l^{(i)}(\mathbf{w}, b) = \frac{1}{n} \sum_{i=1}^{n} \frac{1}{2} \left( \mathbf{w}x^{(i)} + b - y^{(i)} \right)^2
 $

 * When training the model, we seek parameter $(w*,b*)$ that minimizes the total loss across all training examples:
 $\mathbf{w}^*, b^* = \underset{\mathbf{w}, b}{\mathrm{argmin}} \ L(\mathbf{w}, b)$



In [4]:
##defining loss function
# Convert y_hat to a numpy array for element-wise operations
y_hat_np = np.array(y_hat)

# Calculate the squared error for each prediction and then take the mean
# according to the formula L(w, b) = (1/n) * sum( (y_hat - y_true)^2 )
# The 0.5 factor is often included in the definition of MSE for convenience in differentiation.
loss = 0.5 * np.mean((y_hat_np - y_true)**2)

print(f"Average loss: {loss}")

Average loss: 55.125


## Analytic Solution

* We can find the optimal parameters analytically by applyinf a simple formula as folows.
* First, we can subsume that bias `b` into the parameter `w` by appending a column to the design matrix consists of all 1s. Then our prediction problem is to minimize $|| y -Xw||^2$.
* As long as the matrix X has full rank that is no feature is linearly dependent on the others, then there wil be just one critical point on the loss surface and it corresponds to the minimum of the loss over the entire domain.
* Taking the derivative of the loss with respect to `w` and setting it equal to zero yields:
 $\frac{\partial}{\partial \mathbf{w}} \|\mathbf{y} - \mathbf{Xw}\|^2 = 2\mathbf{X}^\top (\mathbf{Xw} - \mathbf{y}) = 0$ hence $X^Ty = X^TXw$.

# Minibatch Stochastic Gradient Descent

* The key technique for optimizing nearly every deep learning model, consists of iteratively reducing the error by updating the parameters in the direction that incrementally lowers the loss function.

* This algorithm is called `gradient descent`.
* Think of it as a hiker trying to find the lowest point in a mountaineous valley, while being sorrounded by fog, they can't see the bottom but they can feel the sloe of ground under their feet and take a step downhill.


## Variants of gradients Descent

1. Stochastic Gradient Descent: Here the SGD  algorithm looks at only one random training example, calculates loss, and updates parameters immediately.
* Cons:
  * 1. It is noisy and has unstable updates - SGD uses one samle at a time. This makes the gradient estimate noisy(updates are incosistent and fluctuate a lot) which results to making convergence unsmooth.
  * 2. Slow convergence at exact minimum- near the optimum,SGD keeps oscillating instead of settling which in turn makes it never converge, and only hover around the minimum.
  * 3. It's highly sensitive to learning rate i.e if the learning rate is too large it divergence away from the optimal minimum, if it's too small it will have an extremely slow learning.

* 2. Minibatch Stochastic Gradient Descent- here instead of using one example or the whole dataset,it uses small groups of examples (Minibatchs) typically between 32 and 512.
  * Advantages:
     1. It is less noisy than SGD since it uses small batches instead of one sample hence making gradient estimates more stable which in turn makes the loss curve smoother.
     2. It is faster than batch gradient descent since parameters are updated multiple times per epoch.
    3. It ensures efficient use of hardware i.e TPU/GPUs since vectorized operations on minibatchs are highly optimized
* Let's implement a code version of minibatch gradient descent.

In [17]:
import numpy as np
x = np.arange(1,9)
y_true = 2*x

# Initialize parameters as floats for accurate updates
weight = 2.0
bias = 1.0
learning_rate = 0.01
minibatch_size = 4
epochs = 100

# Prepare data batches
x_batches = np.array_split(x, len(x) // minibatch_size)
y_true_batches = np.array_split(y_true, len(y_true) // minibatch_size)

print(f"Initial Weight: {weight:.4f}, Bias: {bias:.4f}")

# Training loop
for epoch in range(epochs):
  total_epoch_loss = 0.0
  for x_batch, y_batch in zip(x_batches, y_true_batches):
    # 1. Make prediction with current weights and bias
    y_hat_batch = weight * x_batch + bias

    # Calculate the error term
    error = y_hat_batch - y_batch

    # 2. Calculate batch loss (Mean Squared Error)
    batch_loss = np.mean(0.5 * (error**2)) # Using 0.5 * squared error as defined
    total_epoch_loss += batch_loss

    # 3. Calculate gradients (mean of the derivatives for the batch)
    # As per the loss function L = (1/n) * sum(0.5 * (y_hat - y)^2)
    # dL/dw = (1/n) * sum((y_hat - y) * x)
    # dL/db = (1/n) * sum(y_hat - y)
    grad_w = np.mean(error * x_batch)
    grad_b = np.mean(error)

    # 4. Update parameters
    weight = weight - learning_rate * grad_w
    bias = bias - learning_rate * grad_b

  # Print average loss for the epoch to monitor progress
  if (epoch + 1) % 10 == 0 or epoch == 1: # Prints every 10 epochs or first epoch
      avg_epoch_loss = total_epoch_loss / len(x_batches)
      print(f"Epoch {epoch + 1}: Loss = {avg_epoch_loss:.4f}, Weight = {weight:.4f}, Bias = {bias:.4f}")

print(f"\nFinal Weight: {weight:.4f}, Bias: {bias:.4f}")


Initial Weight: 2.0000, Bias: 1.0000
Epoch 2: Loss = 0.1885, Weight = 1.8818, Bias = 0.9704
Epoch 10: Loss = 0.0972, Weight = 1.8452, Bias = 0.9314
Epoch 20: Loss = 0.0894, Weight = 1.8513, Bias = 0.8934
Epoch 30: Loss = 0.0822, Weight = 1.8574, Bias = 0.8570
Epoch 40: Loss = 0.0757, Weight = 1.8632, Bias = 0.8221
Epoch 50: Loss = 0.0696, Weight = 1.8688, Bias = 0.7886
Epoch 60: Loss = 0.0641, Weight = 1.8741, Bias = 0.7564
Epoch 70: Loss = 0.0590, Weight = 1.8793, Bias = 0.7256
Epoch 80: Loss = 0.0543, Weight = 1.8842, Bias = 0.6961
Epoch 90: Loss = 0.0499, Weight = 1.8889, Bias = 0.6677
Epoch 100: Loss = 0.0459, Weight = 1.8934, Bias = 0.6405

Final Weight: 1.8934, Bias: 0.6405


## 1.2 Vectorization for Speed

* When training our models, we typically want to process whole minibathces of examples simultaneously.
* Doing this requires that we vectorixe the calculations and leverage fast linear algebra libraries rather than writing costly-loops in Python.

In [21]:
##example
n = 10000 #n-dimensional vector
a = torch.ones(n)
b = torch.ones(n)


In [22]:
##testing how a for-loop will take
c = torch.zeros(n)
t = time.time()
for i in range(n):
  c[i] = a[i] + b[i]
f'{time.time()-t:.5f} sec'

'0.17138 sec'

In [23]:
##using operator + for elementwise sum
t = time.time()
d = a+b
f"{time.time() -t:.5f} sec"

'0.00050 sec'

* From above we can conclude that using the operator `+` is faster than the for-loop.